In [3]:
!wget https://github.com/Panizzutti/GPT_from_scratch/blob/83c554974ce0c1904d70d8133a753f86a66611d2/divina_commedia.txt

--2025-02-16 10:41:15--  https://github.com/Panizzutti/GPT_from_scratch/blob/83c554974ce0c1904d70d8133a753f86a66611d2/divina_commedia.txt
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘divina_commedia.txt.1’

divina_commedia.txt     [ <=>                ]   1.19M  --.-KB/s    in 0.09s   

2025-02-16 10:41:16 (13.8 MB/s) - ‘divina_commedia.txt.1’ saved [1248086]



In [20]:
import torch

In [5]:
with open('divina_commedia.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()


In [6]:
print(text[:200])







<!DOCTYPE html>
<html
  lang="en"
  
  data-color-mode="auto" data-light-theme="light" data-dark-theme="dark"
  data-a11y-animated-images="system" data-a11y-link-underlines="true"
  
  >



  <h


In [7]:
len(text)

1241594

In [8]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)



 !"#%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]_`abcdefghijklmnopqrstuvwxyz{}~ ·àèéìïòóù’…
105


In [9]:
char_to_int = { ch: i for i,ch in enumerate(chars)}
int_to_char = { i: ch for i,ch in enumerate(chars)}
encode = lambda s: [char_to_int[c] for c in s] #encoder function
decode = lambda l: "".join(int_to_char[i] for i in l) #decoder function

#not byte pair encoding, for simplicity

In [10]:
decode(encode("cianoxxj"))


'cianoxxj'

In [11]:
encode("ciao mondo")

[66, 72, 64, 78, 1, 76, 78, 77, 67, 78]

In [12]:
data = torch.tensor(encode(text))

In [13]:
n = int(0.9 * len(data))
train_data= data[:n]
val_data= data[n:]


In [14]:
batch_size = 4
block_size= 10

train_data[:block_size+1]

tensor([ 0,  0,  0,  0,  0,  0, 28,  2, 36, 47, 35])

In [15]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 10])
tensor([[60,  3, 14, 60, 81,  3, 12,  3,  1,  1],
        [20, 13, 14, 18, 20, 19, 13, 17, 14, 24],
        [15, 48, 64, 77, 72, 89, 89, 84, 83, 83],
        [12,  3, 80, 84, 72, 85, 72,  1, 75, 84]])
targets:
torch.Size([4, 10])
tensor([[ 3, 14, 60, 81,  3, 12,  3,  1,  1, 37],
        [13, 14, 18, 20, 19, 13, 17, 14, 24, 20],
        [48, 64, 77, 72, 89, 89, 84, 83, 83, 72],
        [ 3, 80, 84, 72, 85, 72,  1, 75, 84, 76]])
----
when input is [60] the target: 3
when input is [60, 3] the target: 14
when input is [60, 3, 14] the target: 60
when input is [60, 3, 14, 60] the target: 81
when input is [60, 3, 14, 60, 81] the target: 3
when input is [60, 3, 14, 60, 81, 3] the target: 12
when input is [60, 3, 14, 60, 81, 3, 12] the target: 3
when input is [60, 3, 14, 60, 81, 3, 12, 3] the target: 1
when input is [60, 3, 14, 60, 81, 3, 12, 3, 1] the target: 1
when input is [60, 3, 14, 60, 81, 3, 12, 3, 1, 1] the target: 37
when input is [20] the target: 13
when in

In [16]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(42)

class BigramLanguageModel(nn.Module): #this simply calculate the probability of next token only on the previous token

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) #reshape
            targets = targets.view(B*T) #reshape in a single vector
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
m

BigramLanguageModel(
  (token_embedding_table): Embedding(105, 105)
)

In [17]:

logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([40, 105])
tensor(4.9411, grad_fn=<NllLossBackward0>)


CWScòAUnv/A@~>XLà>#XP-i-`d![U69
7l,,Ye=ì]yJì[Q& JdG6o~ï%szù·#XaOF3ok…ogxX +["NE,>>}A;’H-b
 7àV'T&Y5


In [18]:
%%time

optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

for steps in range(20000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.584106922149658
CPU times: user 22.6 s, sys: 641 ms, total: 23.3 s
Wall time: 24.4 s


In [19]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=400)[0].tolist()))


 checo s-ce.25v></di l  t"," stria-ch'mela,\rct_s=" ssta girale-to int-r">1  sfis-21.25fser",[],[]," no liv>
 raretret-le v>
 qure so   iapià er  re\"l sin sereneri\"diopile on pxt'allo  G13.5Va puta\ris=" spr 149",\",[],"per 06 e-be:{mba, .28","Ne e la2<delonga 'a-31"," codacurenfutalipaldi;\rattofronttiar" follch foui clefe mocolinasì chto," bylline-cosea  5lo.5forosì Gla pre-che .6"cheso\r"\r",
